# CUDA编程模型---错误检测，事件及存储单元

#### 通过之前的学习，我们已经初步掌握了利用GPU加速应用程序的方法。接下来，我们针对更多细节加以训练。本次实验课将会介绍：  
1. Cuda编程模型中的错误检测
2. Cuda编程模型中的事件
3. Cuda编程模型中多种类型的存储单元
4. 利用共享存储单元来加速矩阵乘法


#### 1.Cuda编程模型中的错误检测

可以查看Cuda error的四个函数：  
```C++
__host__​__device__​const char* 	cudaGetErrorName ( cudaError_t error )
Returns the string representation of an error code enum name.  

__host__​__device__​const char* 	cudaGetErrorString ( cudaError_t error )
Returns the description string for an error code.  

__host__​__device__​cudaError_t 	cudaGetLastError ( void )
Returns the last error from a runtime call.  

__host__​__device__​cudaError_t 	cudaPeekAtLastError ( void )
Returns the last error from a runtime call.  
```

这里我们采用第二个，并将其封装在error.cuh文件中：
```C++
#pragma once
#include <stdio.h>

#define CHECK(call)                                   \
do                                                    \
{                                                     \
    const cudaError_t error_code = call;              \
    if (error_code != cudaSuccess)                    \
    {                                                 \
        printf("CUDA Error:\n");                      \
        printf("    File:       %s\n", __FILE__);     \
        printf("    Line:       %d\n", __LINE__);     \
        printf("    Error code: %d\n", error_code);   \
        printf("    Error text: %s\n",                \
            cudaGetErrorString(error_code));          \
        exit(1);                                      \
    }                                                 \
} while (0)
```

那么我们就可以在代码中这么使用：
```C++
    CHECK(cudaMemcpy(d_b, h_b, sizeof(int)*n*k, cudaMemcpyHostToDevice));
```

接下来，大家在之前做的[matrix_mul.cu](matrix_mul.cu)文件，添加CHECK()，如果遇到麻烦，请参考[result1.cu](result1.cu)

修改Makefile文件，编译程序

In [39]:
!make

/usr/local/cuda/bin/nvcc -arch=compute_80 -code=sm_80  matrix_mul.cu -o ./matrix_mul


执行查看效果

In [40]:
!./matrix_mul

Pass!!!


修改
```C++
CHECK(cudaMemcpy(h_c, d_c, (sizeof(int)*m*k), cudaMemcpyDeviceToHost)); 
```
成
```C++
CHECK(cudaMemcpy(h_c, d_c, (sizeof(int)*m*k*2), cudaMemcpyDeviceToHost));
```

再编译一下，并执行查看结果

In [28]:
!make

/usr/local/cuda/bin/nvcc -arch=compute_80 -code=sm_80  matrix_mul.cu -o ./matrix_mul


In [30]:
!./matrix_mul

CUDA Error:
    File:       matrix_mul.cu
    Line:       81
    Error code: 1
    Error text: invalid argument


In [32]:
# !sudo /usr/local/cuda/bin/nvprof ./matrix_mul
!/usr/local/cuda/bin/nsys nvprof --print-api-trace  ./matrix_mul

The --print-api-trace  switch is ignored by nsys.

CUDA Error:
    File:       matrix_mul.cu
    Line:       81
    Error code: 1
    Error text: invalid argument
Generating '/tmp/nsys-report-1d0b.qdstrm'
[1/7] [========================100%] report1.nsys-rep
[2/7] [========================100%] report1.sqlite
[3/7] Executing 'nvtxsum' stats report
SKIPPED: /mnt/CUDA_on_ARM/03_3.4/report1.sqlite does not contain NV Tools Extension (NVTX) data.
[4/7] Executing 'cudaapisum' stats report

CUDA API Statistics:

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)   Med (ns)  Min (ns)  Max (ns)   StdDev (ns)        Name      
 --------  ---------------  ---------  ----------  --------  --------  ---------  -----------  ----------------
     99.8        201094896          4  50273724.0    5515.5      4133  201079732  100537338.7  cudaMallocHost  
      0.1           207006          3     69002.0    3972.0      3217     199817     113289.7  cudaMalloc      
      0.0            77393          4   

这时我们就精准的定位了出现错误的文件，位置，以及错误原因

#### 2. Cuda编程模型中的事件。事件的本质就是一个标记，它与其所在的流内的特定点相关联。可以使用时间来执行以下两个基本任务：
- 同步流执行
- 监控设备的进展

流中的任意点都可以通过API插入事件以及查询事件完成的函数，只有事件所在流中其之前的操作都完成后才能触发事件完成。默认流中设置事件，那么其前面的所有操作都完成时，事件才出发完成。
事件就像一个个路标，其本身不执行什么功能，就像我们最原始测试c语言程序的时候插入的无数多个printf一样。

创建和销毁： 

声明:
```C++
cudaEvent_t event;
```
创建：
```C++
cudaError_t cudaEventCreate(cudaEvent_t* event);
```
销毁：
```C++
cudaError_t cudaEventDestroy(cudaEvent_t event);
```
添加事件到当前执行流：
```C++
cudaError_t cudaEventRecord(cudaEvent_t event, cudaStream_t stream = 0);
```
等待事件完成，设立flag：
```C++
cudaError_t cudaEventSynchronize(cudaEvent_t event);//阻塞
cudaError_t cudaEventQuery(cudaEvent_t event);//非阻塞
```
当然，我们也可以用它来记录执行的事件：
```C++
cudaError_t cudaEventElapsedTime(float* ms, cudaEvent_t start, cudaEvent_t stop);
```

接下来，我们就修改matrix_mul.cu程序，来测试一下核函数执行的时间，如果遇到麻烦，请参考[result2.cu](result2.cu)

编译并执行程序

In [44]:
!make

/usr/local/cuda/bin/nvcc -arch=compute_80 -code=sm_80  matrix_mul.cu -o ./matrix_mul


In [45]:
!./matrix_mul

Time = 0.023232 ms.
Pass!!!


In [46]:
#! /usr/local/cuda/bin/nvprof ./matrix_mul

======== Warning: nvprof is not supported on devices with compute capability 8.0 and higher.
                  Use NVIDIA Nsight Systems for GPU tracing and CPU sampling and NVIDIA Nsight Compute for GPU profiling.
                  Refer https://developer.nvidia.com/tools-overview for more details.



In [1]:
# 这里根据自己的环境进行了修改。
!/usr/local/cuda/bin/nsys --print-api-trace ./matrix_mul

Unknown command: --print-api-trace
 usage: nsys [--version] [--help] <command> [<args>] [application] [<application args>]

 The most commonly used nsys commands are:
	profile       Run an application and capture its profile into a QDSTRM file.
	launch        Launch an application ready to be profiled.
	start         Start a profiling session.
	stop          Stop a profiling session and capture its profile into a QDSTRM file.
	cancel        Cancel a profiling session and discard any collected data.
	stats         Generate statistics from an existing nsys-rep or SQLite file.
	status        Provide current status of CLI or the collection environment.
	shutdown      Disconnect launched processes from the profiler and shutdown the profiler.
	sessions list List active sessions.
	export        Export nsys-rep file into another format.
	analyze       Run rules on an existing nsys-rep or SQLITE file.
	nvprof        Translate nvprof switches to nsys switches and execute collection.

 Use 'nsys 

#### 3.Cuda编程模型中的多种存储单元

![gpu_memory](gpu_memory.png)

- 寄存器

寄存器是速度最快的存储单元，位于GPU芯片的SM上，用于存储局部变量。每个SM（SMX）上有成千上万的32位寄存器，当kernel函数启动后，这些寄存器被分配给指定的线程来使用。由于不同kernel函数需要的寄存器数量也不相等，所以，也有一个规定一个线程的最大寄存器数量是256个。寄存器的最小单位是register file，所以，在很多图上也会用register file来表示。

- Local Memory

Local Memory本身在硬件中没有特定的存储单元，而是从Global Memory虚拟出来的地址空间。Local Memory是为寄存器无法满足存储需求的情况而设计的，主要是用于存放单线程的大型数组和变量。Local Memory是线程私有的，线程之间是不可见的。由于GPU硬件单位没有Local Memory的存储单元，所以，针对它的访问是比较慢的。从上面的表格中，也可以看到跟Global Memory的访问速度是接近的。

- Shared Memory

Shared Memory位于GPU芯片上，访问延迟仅次于寄存器。Shared Memory是可以被一个Block中的所有Thread来进行访问的，可以实现Block内的线程间的低开销通信。在SMX中，L1 Cache跟Shared Memory是共享一个64KB的告诉存储单元的，他们之间的大小划分不同的GPU结构不太一样；

- Constant Memory

Constant Memory类似于Local Memory，也是没有特定的存储单元的，只是Global Memory的虚拟地址。因为它是只读的，所以简化了缓存管理，硬件无需管理复杂的回写策略。Constant Memory启动的条件是同一个warp所有的线程同时访问同样的常量数据。

- Global Memory

Global Memory在某种意义上等同于GPU显存，kernel函数通过Global Memory来读写显存。Global Memory是kernel函数输入数据和写入结果的唯一来源。

- Texture Memory

Texture Memory是GPU的重要特性之一，也是GPU编程优化的关键。Texture Memory实际上也是Global Memory的一部分，但是它有自己专用的只读cache。这个cache在浮点运算很有用，Texture Memory是针对2D空间局部性的优化策略，所以thread要获取2D数据就可以使用texture Memory来达到很高的性能。从读取性能的角度跟Constant Memory类似。

- Host Memory

主机端存储器主要是内存可以分为两类：可分页内存（Pageable）和页面 （Page-Locked 或 Pinned）内存。

可分页内存通过操作系统 API(malloc/free) 分配存储器空间，该内存是可以换页的，即内存页可以被置换到磁盘中。可分页内存是不可用使用DMA（Direct Memory Acess)来进行访问的，普通的C程序使用的内存就是这个内存。

#### 4.利用Shrared Memory来优化矩阵乘法

![shared_memory](shared_memory.png)

当我们在处理矩阵乘法时，假设矩阵M(m,k)\*N(k,n) = P(m,n)。那么，矩阵M中的一个数值m(x,y),就要被grid中所有满足threadIdx.y+blockIdx.y\*blockDim.y = y的线程从Global Memory中读一次，一共就是K次。那么，我们看到这么多重复读取，就可以把这个变量放在Shared Memory中，极大地减少每次的读取时间。我们可以按照下面的代码来修改martix_mul的核函数：

```C++   
__global__ void gpu_matrix_mult_shared(int *d_a, int *d_b, int *d_result, int m, int n, int k) 
{
    __shared__ int tile_a[BLOCK_SIZE][BLOCK_SIZE];
    __shared__ int tile_b[BLOCK_SIZE][BLOCK_SIZE];

    int row = blockIdx.y * BLOCK_SIZE + threadIdx.y;
    int col = blockIdx.x * BLOCK_SIZE + threadIdx.x;
    int tmp = 0;
    int idx;

    for (int sub = 0; sub < gridDim.x; ++sub) 
    {
        idx = row * n + sub * BLOCK_SIZE + threadIdx.x;
        tile_a[threadIdx.y][threadIdx.x] = row<n && (sub * BLOCK_SIZE + threadIdx.x)<n? d_a[idx]:0;
        idx = (sub * BLOCK_SIZE + threadIdx.y) * n + col;
        tile_b[threadIdx.y][threadIdx.x] = col<n && (sub * BLOCK_SIZE + threadIdx.y)<n? d_b[idx]:0;
        
        __syncthreads();
        for (int k = 0; k < BLOCK_SIZE; ++k) 
        {
            tmp += tile_a[threadIdx.y][k] * tile_b[k][threadIdx.x];
        }
        __syncthreads();
    }
    if(row < n && col < n)
    {
        d_result[row * n + col] = tmp;
    }
}

```

![array_2d](array_2to1.png)

修改[matrix_mul.cu](matrix_mul.cu)文件，利用Makefile编译，并执行。如果遇到困难，请参考[result4.cu](result4.cu)

In [7]:
# 
!make

/usr/local/cuda/bin/nvcc -arch=compute_80 -code=sm_80  matrix_mul.cu -o ./matrix_mul


In [8]:
!./matrix_mul

Time_global = 2.09488 ms.
Time_share = 1.33568 ms.
258892266 
Pass!!!


利用nvprof来查看性能

In [ ]:
!sudo /usr/local/cuda/bin/nvprof ./matrix_mul

课后作业： 

1. 修改BLOCK_SIZE查看性能变化
2. 修改代码，尝试产生bank_conflict

In [33]:
# BLOCK_SIZE = 2
!make
!./matrix_mul

/usr/local/cuda/bin/nvcc -arch=compute_80 -code=sm_80  matrix_mul.cu -o ./matrix_mul
Time_global = 9.65494 ms.
Time_share = 22.9141 ms.
Pass!!!


In [24]:
# BLOCK_SIZE = 4
!make
!./matrix_mul

/usr/local/cuda/bin/nvcc -arch=compute_80 -code=sm_80  matrix_mul.cu -o ./matrix_mul
Time_global = 3.7712 ms.
Time_share = 4.26186 ms.
Pass!!!


In [25]:
# BLOCK_SIZE = 8
!make
!./matrix_mul

/usr/local/cuda/bin/nvcc -arch=compute_80 -code=sm_80  matrix_mul.cu -o ./matrix_mul
Time_global = 2.06074 ms.
Time_share = 1.33578 ms.
Pass!!!


In [32]:
# BLOCK_SIZE = 10
!make
!./matrix_mul

/usr/local/cuda/bin/nvcc -arch=compute_80 -code=sm_80  matrix_mul.cu -o ./matrix_mul
Time_global = 2.34336 ms.
Time_share = 1.63059 ms.
Pass!!!


In [11]:
# BLOCK_SIZE = 16
!make
!./matrix_mul

/usr/local/cuda/bin/nvcc -arch=compute_80 -code=sm_80  matrix_mul.cu -o ./matrix_mul
Time_global = 2.07722 ms.
Time_share = 1.15958 ms.
258892266 
Pass!!!


In [10]:
# BLOCK_SIZE = 20
!make
!./matrix_mul

/usr/local/cuda/bin/nvcc -arch=compute_80 -code=sm_80  matrix_mul.cu -o ./matrix_mul
Time_global = 2.10771 ms.
Time_share = 1.1791 ms.
258892266 
Pass!!!


In [41]:
# BLOCK_SIZE = 32
!make
!./matrix_mul

/usr/local/cuda/bin/nvcc -arch=compute_80 -code=sm_80  matrix_mul.cu -o ./matrix_mul
Time_global = 0.071168 ms.
Time_share = 0.029344 ms.
30977895 
Pass!!!


课后作业：     
2. 修改代码，尝试产生bank_conflict

修改[matrix_mul.cu](matrix_mul.cu)文件，利用Makefile编译，并执行。如果遇到困难，请参考[result5.cu](result5.cu)

In [50]:
# BLOCK_SIZE = 32
!make
!./matrix_mul

/usr/local/cuda/bin/nvcc -arch=compute_80 -code=sm_80  matrix_mul.cu -o ./matrix_mul
Time_global = 1.74858 ms.
Time_share = 1.01632 ms.
Time_share = 3.10592 ms.
260387967 
Pass!!!


In [8]:
import numpy as np
a = np.arange(10).reshape(2,5)
a

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [10]:
a[0][2], a.shape

(2, (2, 5))